## A demo using Hydrogen Hamiltonian with GPT-QE.

In [8]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [9]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor
import logging

logging.getLogger('benchmark.molecule').setLevel(logging.WARNING)

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
molecule = generate_molecule("N", "N", 2.0, "sto-3g", active_orbitals=[4, 5, 6, 7, 8, 9],
                             bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=6)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2],
                  tool='qulacs')


converged SCF energy = -106.871504045608
paulis [+IIIIIXIIIIII, +IIIIIXZYIIII, +IIIIIXZZZYII, +IIIIIXZZZZZY, +IIIIIYZXIIII, +IIIIIYZZZXII, +IIIIIYZZZZZX, +IIIIXIIIIIII, +IIIIXXIIIIXY, +IIIIXXIIIIYX, +IIIIXXIIXYII, +IIIIXXIIYXII, +IIIIXXXYIIII, +IIIIXXYXIIII, +IIIIXYIIIIXX, +IIIIXYIIIIYY, +IIIIXYIIXXII, +IIIIXYIIYYII, +IIIIXYXXIIII, +IIIIXYYYIIII, +IIIIXZYIIIII, +IIIIXZZZYIII, +IIIIXZZZZZYI, +IIIIYXIIIIXX, +IIIIYXIIIIYY, +IIIIYXIIXXII, +IIIIYXIIYYII, +IIIIYXXXIIII, +IIIIYXYYIIII, +IIIIYYIIIIXY, +IIIIYYIIIIYX, +IIIIYYIIXYII, +IIIIYYIIYXII, +IIIIYYXYIIII, +IIIIYYYXIIII, +IIIIYZXIIIII, +IIIIYZZZXIII, +IIIIYZZZZZXI, +IIIXIIIIIIII, +IIIXXIIXYIII, +IIIXXIIYXIII, +IIIXXIXZZYII, +IIIXXIYZZXII, +IIIXYIIXXIII, +IIIXYIIYYIII, +IIIXYIXZZXII, +IIIXYIYZZYII, +IIIXZXIXZYII, +IIIXZXIYZXII, +IIIXZYIXZXII, +IIIXZYIYZYII, +IIIXZZZYIIII, +IIIXZZZZZYII, +IIIXZZZZZZZY, +IIIYXIIXXIII, +IIIYXIIYYIII, +IIIYXIXZZXII, +IIIYXIYZZYII, +IIIYYIIXYIII, +IIIYYIIYXIII, +IIIYYIXZZYII, +IIIYYIYZZXII, +IIIYZXIXZXII, +IIIYZ

## FCI energy by diagonalization

In [10]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-107.43702368448199


In [11]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -106.8715040456087


## Setup for GPT

In [12]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.n_gates = 60  # The number of gates for each circuit
model_config.block_size = model_config.n_gates
model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0.1
model_config.resid_pdrop = 0.1
model_config.attn_pdrop = 0.1
model_config.std = 0.02
model_config.energy_offset = 106
model = GPT(model_config, cost)

number of parameters: 87.49M


In [13]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 500
train_config.num_workers = 10
train_config.n_samples = 50
trainer = Trainer(train_config, model)

running on device mps


In [14]:
file_monitor = FileMonitor()
callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor])
trainer.set_callback('on_batch_end', callback_generator.generate())
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')
print(file_monitor.min_energy)

ImportError: cannot import name 'QuantumStateGpu' from 'qulacs' (/Users/koheinakaji/PycharmProjects/gqe/venv/lib/python3.8/site-packages/qulacs/__init__.py)

In [ ]:
cost.sequence.tool = "qiskit"
indices = file_monitor.min_indices
cost.sequence._get_circuit(indices).qc.draw(output="mpl", plot_barriers=True)